# SIMULIRANO KALJENJE

In [3]:
import random
import math

def initial_solution(A):
    return random.sample(A, len(A))

# broj trojki koje zadovoljavaju uslov za datu permutaciju solutions
def count_triplets(solution, C):
    count = 0
    for (a, b, c) in C:
        index_a = solution.index(a)
        index_b = solution.index(b)
        index_c = solution.index(c)
        if (index_a < index_b < index_c) or (index_c < index_b < index_a):
            count += 1
    return count

def generate_neighbor(solution):
    # generisemo jednu susednu permutaciju da bi dopustili i prihvatanje losijih resenja
    # kako bi izbegli zaglavljivanje u lokalnim optimumima (kao kod lokalne pretrage)
    neighbor = solution[:]
    i, j = random.sample(range(len(solution)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

def simulated_annealing_max_betweenness(A, C, initial_temperature=100, cooling_rate=0.95, stopping_temperature=1e-6, max_iterations=1000):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            print("temperatura")
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        # ako je delta score negativan i visoka temperatura 
        #     -> prihvatice se i to resenje (iako je losije)
        # kako temperatura opada -> manja verovatnoca da cemo da prihvatamo i losija resenja
        #     -> prihvatace se uglavnom samo ona resenja sa delta_score > 0
                                                    # Metropolisov kriterijum
        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature *= cooling_rate
    
    return best_solution, best_score

In [3]:
import math
import random

def initial_solution(A):
    return random.sample(A, len(A))

def count_triplets(solution, C):
    count = 0
    for (a, b, c) in C:
        index_a = solution.index(a)
        index_b = solution.index(b)
        index_c = solution.index(c)
        if (index_a < index_b < index_c) or (index_c < index_b < index_a):
            count += 1
    return count

def generate_neighbor(solution):
    neighbor = solution[:]
    i, j = random.sample(range(len(solution)), 2)
    neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
    return neighbor

# Primeri strategija hlađenja temperature

def geometric_cooling(initial_temperature, iteration, cooling_rate):
    return initial_temperature * cooling_rate ** iteration

def sqrt_cooling(initial_temperature, iteration):
    return initial_temperature / math.sqrt(iteration + 1)

def log_cooling(initial_temperature, iteration):
    return initial_temperature / math.log(iteration + 2)

def adaptive_cooling(initial_temperature, iteration, acceptance_rate, target_acceptance_rate=0.5):
    if acceptance_rate > target_acceptance_rate:
        return initial_temperature / math.log(iteration + 2)
    else:
        return initial_temperature * 0.95

def simulated_annealing(A, C, cooling_strategy, initial_temperature=100, stopping_temperature=1e-6, max_iterations=1000, **kwargs):
    current_solution = initial_solution(A)
    current_score = count_triplets(current_solution, C)
    best_solution = current_solution
    best_score = current_score
    temperature = initial_temperature
    
    for iteration in range(max_iterations):
        if temperature < stopping_temperature:
            break
        
        neighbor = generate_neighbor(current_solution)
        neighbor_score = count_triplets(neighbor, C)
        
        delta_score = neighbor_score - current_score

        if delta_score > 0 or random.uniform(0, 1) < math.exp(delta_score / temperature):
            current_solution = neighbor
            current_score = neighbor_score
        
        if current_score > best_score:
            best_solution = current_solution
            best_score = current_score
        
        temperature = cooling_strategy(initial_temperature, iteration, **kwargs)
    
    return best_solution, best_score

In [11]:
A = [1, 2, 3, 4, 5]
C = [(1, 2, 3), (2, 4, 5), (1, 3, 5)]

# Geometrijsko hladjenje
best_solution, best_score = simulated_annealing(A, C, geometric_cooling, cooling_rate=0.95)
print("Geometrijsko hladjenje - Najbolje resenje:", best_solution)
print("Najbolji rezultat:", best_score)

# Hladjenje kvadratnim korenom
best_solution, best_score = simulated_annealing(A, C, sqrt_cooling)
print("Hladjenje kvadratnim korenom - Najbolje resenje:", best_solution)
print("Najbolji rezultat:", best_score)

# Logaritamsko hladjenje
best_solution, best_score = simulated_annealing(A, C, log_cooling)
print("Logaritamsko hladjenje - Najbolje resenje:", best_solution)
print("Najbolji rezultat:", best_score)

# Adaptivno hladjenje
best_solution, best_score = simulated_annealing(A, C, adaptive_cooling, acceptance_rate=0.6)
print("Adaptivno hladjenje - Najbolje resenje:", best_solution)
print("Najbolji rezultat:", best_score)

Geometrijsko hladjenje - Najbolje resenje: [1, 2, 3, 4, 5]
Najbolji rezultat: 3
Hladjenje kvadratnim korenom - Najbolje resenje: [5, 4, 3, 2, 1]
Najbolji rezultat: 3
Logaritamsko hladjenje - Najbolje resenje: [1, 2, 4, 3, 5]
Najbolji rezultat: 3
Adaptivno hladjenje - Najbolje resenje: [5, 3, 4, 2, 1]
Najbolji rezultat: 3


Kada je temperatura visoka, Metropolisov kriterijum je veći, što znači da je veća verovatnoća prihvatanja lošijeg rešenja. Kako temperatura opada tokom vremena, verovatnoća prihvatanja lošijeg rešenja takođe opada, što omogućava algoritmu da se postepeno fokusira na poboljšavanje kvaliteta rešenja.

In [111]:
A = list(range(1, 16))
C = [(2, 1, 3), (2, 1, 4), (1, 2, 4), (4, 2, 1), (1, 2, 3)]
solution, score = simulated_annealing_max_betweenness(A, C)
print("Najbolje rešenje:", solution)
print("Broj trojki koje zadovoljavaju uslove:", score)


temperatura
Najbolje rešenje: [11, 3, 7, 5, 6, 12, 10, 8, 4, 2, 15, 14, 9, 1, 13]
Broj trojki koje zadovoljavaju uslove: 3


In [103]:
A = [1, 2, 3, 4, 5, 6, 7, 8, 9]
C = [
    (2, 1, 3), (2, 1, 4), (1, 2, 4), (4, 2, 1), (1, 2, 3),
    (3, 2, 4), (1, 4, 2), (5, 2, 3), (3, 1, 5), (4, 5, 2),
    (6, 3, 2), (4, 6, 1), (7, 2, 6), (3, 7, 1), (8, 1, 2),
    (2, 3, 8), (9, 2, 1), (1, 9, 3)
]

solution, score = simulated_annealing_max_betweenness(A, C)
print("Najbolje rešenje:", solution)
print("Broj trojki koje zadovoljavaju uslove:", score)


temperatura
Najbolje rešenje: [4, 9, 6, 5, 2, 1, 7, 3, 8]
Broj trojki koje zadovoljavaju uslove: 13


In [51]:
# Skup A sa 7 elemenata
A = [1, 2, 3, 4, 5, 6, 7]

# Kolekcija C sa 14 trojki
C = [
    (2, 1, 3), (2, 1, 4), (1, 2, 4), (4, 2, 1), (1, 2, 3),
    (3, 2, 4), (1, 4, 2), (5, 2, 3), (3, 1, 5), (4, 5, 2),
    (6, 3, 2), (4, 6, 1), (7, 2, 6), (3, 7, 1)
]


solution, score = simulated_annealing_max_betweenness(A, C)
print("Najbolje rešenje:", solution)
print("Broj trojki koje zadovoljavaju uslove:", score)


Najbolje rešenje: [4, 5, 6, 2, 1, 7, 3]
Broj trojki koje zadovoljavaju uslove: 10


In [37]:
# Generisanje skupa A sa 100 elemenata
A = list(range(1, 101))
C = [(random.choice(A), random.choice(A), random.choice(A)) for _ in range(200)]

temperatura
Najbolje rešenje: [41, 42, 55, 57, 91, 44, 94, 22, 78, 4, 82, 23, 76, 93, 88, 73, 24, 77, 50, 99, 10, 34, 49, 56, 13, 64, 27, 72, 20, 70, 31, 30, 36, 43, 67, 96, 1, 74, 14, 47, 61, 63, 52, 66, 97, 62, 100, 15, 32, 46, 11, 80, 84, 89, 21, 2, 33, 45, 3, 25, 51, 17, 59, 35, 53, 71, 6, 29, 75, 69, 39, 18, 87, 58, 37, 8, 28, 7, 81, 5, 92, 26, 85, 48, 83, 90, 38, 54, 86, 9, 98, 79, 65, 16, 95, 60, 68, 19, 12, 40]
Broj trojki koje zadovoljavaju uslove: 125


In [5]:
# izgenerisan skup A sa 100 elemenata
A = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
C = [(60, 2, 24), (60, 61, 23), (54, 10, 11), (15, 76, 23), (89, 39, 69), (75, 20, 90), (52, 56, 94), (72, 28, 69), (72, 4, 14), (15, 44, 91), (5, 76, 95), (19, 4, 61), (52, 66, 44), (71, 27, 62), (58, 3, 90), (34, 79, 96), (20, 73, 67), (55, 82, 52), (62, 90, 90), (89, 38, 21), (13, 94, 2), (74, 55, 41), (72, 15, 83), (21, 70, 94), (25, 66, 23), (29, 20, 96), (3, 54, 88), (1, 54, 6), (44, 13, 34), (13, 61, 69), (31, 72, 58), (38, 66, 65), (34, 73, 1), (21, 13, 29), (100, 78, 65), (20, 37, 54), (44, 23, 42), (49, 34, 45), (41, 7, 34), (39, 14, 69), (38, 63, 15), (79, 38, 44), (30, 62, 15), (66, 69, 80), (5, 72, 99), (78, 1, 69), (71, 30, 88), (31, 84, 10), (13, 47, 16), (77, 91, 55), (15, 7, 37), (17, 63, 32), (61, 32, 47), (30, 7, 35), (51, 34, 78), (20, 22, 93), (44, 8, 63), (4, 84, 9), (20, 12, 35), (85, 38, 58), (7, 51, 92), (36, 25, 58), (37, 25, 63), (17, 17, 9), (1, 70, 84), (15, 35, 19), (84, 67, 32), (9, 49, 85), (81, 91, 99), (78, 1, 10), (85, 71, 5), (85, 31, 48), (90, 49, 69), (28, 24, 58), (18, 79, 39), (52, 70, 7), (28, 61, 76), (54, 77, 78), (85, 72, 49), (62, 62, 85), (65, 40, 17), (92, 5, 23), (55, 65, 82), (69, 52, 81), (91, 21, 54), (33, 90, 100), (80, 93, 84), (51, 51, 52), (21, 99, 96), (47, 57, 40), (94, 40, 67), (38, 99, 41), (96, 37, 58), (30, 44, 49), (59, 98, 61), (8, 25, 53), (39, 32, 76), (70, 64, 56), (14, 68, 81), (37, 16, 53), (34, 52, 58), (33, 76, 27), (45, 32, 42), (91, 13, 8), (96, 76, 66), (39, 59, 96), (64, 59, 63), (78, 12, 73), (18, 35, 12), (20, 90, 86), (21, 28, 46), (94, 63, 70), (13, 97, 71), (61, 96, 88), (76, 30, 87), (37, 43, 24), (77, 6, 66), (7, 2, 82), (67, 24, 78), (8, 56, 87), (76, 46, 52), (38, 37, 24), (95, 6, 68), (16, 31, 50), (48, 34, 45), (49, 51, 91), (15, 70, 91), (76, 34, 44), (31, 61, 24), (82, 37, 49), (61, 85, 74), (27, 34, 23), (37, 90, 58), (19, 78, 25), (8, 58, 72), (74, 91, 19), (52, 69, 86), (29, 89, 24), (62, 83, 21), (48, 23, 42), (16, 46, 25), (35, 54, 96), (2, 84, 12), (1, 69, 12), (83, 8, 47), (11, 94, 93), (53, 97, 4), (19, 45, 69), (91, 74, 79), (72, 92, 36), (54, 34, 3), (17, 40, 66), (63, 89, 92), (35, 26, 69), (77, 60, 37), (93, 74, 28), (49, 91, 55), (50, 1, 49), (80, 28, 90), (15, 56, 83), (7, 43, 83), (74, 91, 26), (11, 46, 7), (28, 86, 81), (2, 50, 81), (65, 47, 8), (56, 44, 6), (87, 71, 43), (82, 27, 77), (63, 40, 2), (12, 20, 75), (2, 60, 47), (1, 38, 23), (1, 13, 17), (7, 86, 1), (92, 91, 25), (100, 35, 5), (24, 17, 8), (38, 77, 56), (22, 5, 58), (9, 89, 73), (81, 100, 91), (93, 45, 28), (80, 77, 15), (61, 74, 59), (46, 88, 72), (43, 14, 44), (94, 36, 59), (16, 41, 25), (84, 33, 14), (4, 58, 7), (82, 97, 1), (75, 35, 20), (38, 26, 9), (7, 68, 20), (20, 91, 59), (80, 31, 8), (9, 75, 74), (4, 71, 100), (37, 32, 79)]

solution, score = simulated_annealing_max_betweenness(A, C)
print("Najbolje rešenje:", solution)
print("Broj trojki koje zadovoljavaju uslove:", score)

temperatura
Najbolje rešenje: [28, 24, 53, 61, 87, 74, 45, 11, 85, 71, 15, 78, 39, 97, 27, 32, 42, 73, 30, 95, 76, 89, 16, 75, 22, 77, 94, 63, 17, 46, 36, 25, 56, 5, 8, 79, 92, 18, 80, 31, 14, 50, 7, 6, 68, 54, 1, 96, 57, 38, 47, 40, 35, 34, 55, 90, 86, 43, 64, 93, 51, 62, 81, 100, 33, 83, 20, 41, 49, 70, 67, 21, 13, 52, 29, 2, 65, 91, 60, 26, 4, 88, 3, 69, 58, 66, 99, 84, 10, 44, 98, 12, 48, 82, 72, 19, 23, 59, 9, 37]
Broj trojki koje zadovoljavaju uslove: 125
